In [8]:
import json
import os
import subprocess
import sys
import datetime
from typing import Any, Dict, List, Optional, Type, Union

def run_with_config(
    script_path: str,
    class_or_function: Optional[str] = None,
    method: Optional[str] = None,
    base_dir: str = '.',
    config_filename: str = 'config.json',
    **kwargs
) -> int:
    """
    Универсальная функция для запуска Python-скрипта с параметрами через конфигурационный файл.
    
    Args:
        script_path: Путь к Python-скрипту, который нужно запустить
        class_or_function: Имя класса или функции для вызова (опционально)
        method: Имя метода класса для вызова (для классов)
        base_dir: Базовая директория для сохранения конфигурационного файла
        config_filename: Имя файла конфигурации
        **kwargs: Любые параметры, которые нужно передать в скрипт
        
    Returns:
        Код возврата процесса (0 - успешное выполнение)
    """
    # Конвертируем все datetime объекты в строки
    processed_kwargs = {}
    for k, v in kwargs.items():
        if isinstance(v, datetime.date) or isinstance(v, datetime.datetime):
            processed_kwargs[k] = v.isoformat().split('T')[0]  # Формат YYYY-MM-DD
        else:
            processed_kwargs[k] = v
    
    # Создаем конфигурацию
    config = {
        "params": processed_kwargs
    }
    
    # Добавляем информацию о классе и методе, если указаны
    if class_or_function:
        config["target"] = class_or_function
    if method:
        config["method"] = method
    
    # Полный путь к конфигурационному файлу
    config_path = os.path.join(base_dir, config_filename)
    
    # Сохраняем конфигурацию в файл
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    
    print(f"Конфигурация сохранена в {config_path}")
    
    # Проверяем существование скрипта
    if not os.path.exists(script_path):
        raise FileNotFoundError(f"Скрипт {script_path} не найден")
    
    # Запускаем скрипт с указанием файла конфигурации
    command = [sys.executable, script_path, '--config', config_path]
    
    print("Запуск команды:", " ".join(command))
    
    # Запускаем процесс и перенаправляем вывод
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, 
                              universal_newlines=True, bufsize=1)
    
    # Выводим результаты в реальном времени
    for line in process.stdout:
        print(line, end='')
    
    # Ждем завершения процесса
    process.wait()
    
    if process.returncode == 0:
        print("\nПроцесс успешно завершен")
    else:
        print(f"\nОшибка при выполнении процесса, код: {process.returncode}")
    
    return process.returncode


In [9]:
import tinkoff.invest
from tinkoff.invest import PortfolioRequest, PortfolioPosition, Client, RequestError, CandleInterval, HistoricCandle, \
    OrderType, OrderDirection, Quotation, InstrumentIdType, InstrumentStatus
from tinkoff.invest.services import InstrumentsService
from datetime import datetime
import pandas as pd
import os
import sys
import requests
import time
import zipfile
import numpy as np
import importlib
from concurrent.futures import ThreadPoolExecutor

sys.path.append('/Users/aeshef/Documents/GitHub/kursach/pys')

#####

import market_data
importlib.reload(market_data)
from market_data import MarketDataManager

######

import fundamental_data
importlib.reload(fundamental_data)
from fundamental_data import FinamFinancialParser

######

import tech_analysis
importlib.reload(tech_analysis)
from tech_analysis import TechAnalysisPipeline


######

import news_pipeline
importlib.reload(news_pipeline)
from news_pipeline import NewsPipeline

## Globals

In [19]:
token = "t.GVEetX09IVL9qECDUhjThfpg5TrE2Oxw9cUnAIOxcHh35M2sv1YMy9LBfkt73F3WlV_Ik08etZOVrUNI8StLBQ"
kursach_directory_path = "/Users/aeshef/Documents/GitHub/kursach"
your_directory_path = "/Users/aeshef/Documents/GitHub/kursach/data"

tickers = pd.read_csv(f"{your_directory_path}/tickers.csv")
tickers_list = [
    'SBER',  # Сбербанк
    'GAZP',  # Газпром
    'LKOH',  # Лукойл
    'GMKN',  # ГМК "Норильский никель"
    'ROSN',  # Роснефть
    'TATN',  # Татнефть
    'MTSS',  # МТС
    'ALRS',  # АК Алроса
    'SNGS',  # Сургутнефтегаз
    'VTBR',  # ВТБ
    'NVTK',  # Новатэк
    'MVID',  # М.Видео
    'PHOR',  # ФосАгро
    'SIBN',  # Сибнефть
    'AFKS',  # АФК Система
    'MAGN',  # Магнитогорский металлургический комбинат
    'RUAL'   # Русал
]

start_date="2025-04-20"
end_date="2025-04-21"
timeframe="1h"

YOUR_API_ID = 28994010
YOUR_API_HASH = "6e7a57fdfe1a10b0a3434104b42badf2"

## Initial parser

In [11]:
# manager = MarketDataManager(token, your_directory_path)

# multi_ticker_data = manager.get_data_for_multiple_tickers(
#     tickers=tickers_list, 
#     start_date=start_date, 
#     end_date=end_date, 
#     timeframe=timeframe
# )

## Fundamental

In [12]:
import logging

for ticker in tickers_list:
    logging.info(f"\n========== Processing {ticker} ==========\n")
    parser = FinamFinancialParser(ticker)   # ПЕРЕДАЁШЬ тикер!
    financial_data_df = parser.parse_financial_data()
    if financial_data_df is not None:
        parser.save_to_csv(financial_data_df)
        logging.info(f"Финансовые данные для {ticker} сохранены успешно")
    else:
        logging.warning(f"No data found for {ticker}. Проверь логи, html и soup-просмотр!")

2025-04-21 21:59:42,760 [INFO] 
========== Processing SBER ==========

2025-04-21 21:59:42,761 [INFO] Подгружаем страницу https://www.finam.ru/quote/moex/sber/financial/
2025-04-21 21:59:42,890 [INFO] HTTP STATUS: 200
2025-04-21 21:59:42,895 [INFO] Первые 1000 символов парсинга:
<!DOCTYPE html>
<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <noscript>
   <meta content="0; url=/exhkqyad" http-equiv="refresh"/>
  </noscript>
 </head>
 <style>
  .container {
      display: flex;
      align-content: center;
      align-items: center;
      justify-content: center;
      flex-direction: column;
      min-height: 100vh;
    }
    .load {
      color: grey;
      font-size: 3vh;
    }
    .spinner {
      display: block;
      background:  url('
2025-04-21 21:59:42,895 [INFO] Найдено таблиц: 0
2025-04-21 21:59:42,896 [WARNING] Не найдено ни одной пригодной таблицы!
2025-04-21 21:59:42,896 [WARNING] No data found for SBER. Проверь логи, html и soup-про

## Tech

In [13]:
# pipeline = TechAnalysisPipeline(base_dir="/Users/aeshef/Documents/GitHub/kursach")
# pipeline.run_pipeline(
    
# )

## News

In [21]:
import datetime

run_with_config(
    script_path="/Users/aeshef/Documents/GitHub/kursach/pys/news_pipeline.py",
    class_or_function="NewsPipeline",
    method="run_pipeline",
    base_dir=kursach_directory_path,
    config_filename="news_pipeline_config.json",
    
    tickers=['SBER'],
    collect_telegram=True,
    telegram_api_id=YOUR_API_ID,
    telegram_api_hash=YOUR_API_HASH,
    telegram_channel="cbrstocks",
    telegram_limit=1,
    start_date=start_date,
    end_date=end_date,
    use_cached_telegram=True
)


Конфигурация сохранена в /Users/aeshef/Documents/GitHub/kursach/news_pipeline_config.json
Запуск команды: /Library/Developer/CommandLineTools/usr/bin/python3 /Users/aeshef/Documents/GitHub/kursach/pys/news_pipeline.py --config /Users/aeshef/Documents/GitHub/kursach/news_pipeline_config.json
Запуск pipeline.run_pipeline с параметрами: {'tickers': ['SBER'], 'collect_telegram': True, 'telegram_api_id': 28994010, 'telegram_api_hash': '6e7a57fdfe1a10b0a3434104b42badf2', 'telegram_channel': 'cbrstocks', 'telegram_limit': 1, 'start_date': datetime.date(2025, 4, 20), 'end_date': datetime.date(2025, 4, 21), 'use_cached_telegram': True}
2025-04-21 22:16:29 - NewsPipeline - INFO - Запуск пайплайна анализа новостей для 1 тикеров
2025-04-21 22:16:29 - NewsPipeline - INFO - Диапазон дат: 2025-04-20 - 2025-04-21
2025-04-21 22:16:29 - pymorphy2.opencorpora_dict.wrapper - INFO - Loading dictionaries from /Users/aeshef/Library/Python/3.9/lib/python/site-packages/pymorphy2_dicts_ru/data
2025-04-21 22:16:

KeyboardInterrupt: 